Application in which we read and process altimetry measurements.

In [1]:
import os
import glob
import logging
import shutil
import subprocess
import numpy as np
import importlib
import sys
from matplotlib import rcParams
from mpl_toolkits.basemap import Basemap
import pydiva2d
import netCDF4
import matplotlib.pyplot as plt

If you get a `ModuleNotFoundError` message, ensure that the module `pydiva2d` is available in the Python path.

# Setup 

## Logging configuration

The *logging* is already configured in `pydiva2d`.<br>
Replace 'DEBUG' by 'INFO', 'WARNING' or 'ERROR'.

In [2]:
logger = logging.getLogger('divaAltimetry')
logger.setLevel(logging.WARNING)

## Files and directories

Set the path to the Diva installation you want and the main input files.   
A different file will be used for the mesh generation.

In [3]:
divadir = "/home/ctroupin/Software/DIVA/DIVA-diva-4.7.1"
projectdir = "/home/ctroupin/Projects/Altimetry-Interpolation/"
coastfile = os.path.join(projectdir, "diva/coast.cont")
datadir = os.path.join(projectdir, "data/")
paramfile = os.path.join(projectdir, "diva/param.par")
paramfilemesh = os.path.join(projectdir, "diva/param.par.mesh")
outputdir = os.path.join(projectdir, "results/diva2D/")
if not os.path.exists(outputdir):
    os.makedirs(outputdir)

## Matplotlib options

In [4]:
#rcParams.update({'font.size': 16, 'figure.dpi': 300, 'savefig.bbox': 'tight'})
# rcParams.keys

### Projection
Will be used for the plots.

In [5]:
m = Basemap(projection='merc',
            llcrnrlon=-6., llcrnrlat=30.,
            urcrnrlon=40., urcrnrlat=48.,
            lat_ts=39., resolution='i')

## Directories
Create variables storing the Diva directories and files.

In [6]:
DivaDirs = pydiva2d.DivaDirectories(divadir)
DivaFiles = pydiva2d.Diva2Dfiles(DivaDirs.diva2d)

# Prepare the input files
## Contour
We copy the existing file (created using the local topography) to the Diva input directory.

In [7]:
shutil.copy2(coastfile, DivaFiles.contour)

'/home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/DIVA3D/divastripped/input/coast.cont'

## Parameters and mesh
We first copy the parameter file used for the mesh generation.   
We can then perform the mesh generation, before processing all the data files.

In [8]:
shutil.copy2(paramfilemesh, DivaFiles.parameter)
pydiva2d.Diva2DMesh().make(divadir)

In [9]:
eee = pydiva2d.Diva2DMesh()

In [14]:
eee.read_from_np(DivaFiles.mesh, DivaFiles.meshtopo)

IndexError: index 6706 is out of bounds for axis 0 with size 6704

## Analysis
We use another parameter file, loop over the data files and copy the outputs (netCDF) in the specified directory.

In [12]:
shutil.copy2(paramfile, DivaFiles.parameter)
for datafile in sorted(glob.glob(os.path.join(datadir, 'data_201409*.dat'))):
    
    # Copy data file
    logger.info("Working on data file {0}".format(os.path.basename(datafile)))
    shutil.copy2(datafile, DivaFiles.data)
    
    # Perform analysis and copy results
    pydiva2d.Diva2DResults().make(divadir)
    outputfile = "".join((os.path.basename(datafile).split('.')[0],'.nc'))
    shutil.copy2(DivaFiles.result, os.path.join(outputdir, outputfile))
    
    # Make plot
    #divadata = pydiva2d.Diva2DData().read_from(datafile)

In [ ]:
divadata.x

# Time for some plots
Let's display:
* the observations (colored dots), 
* the output grid,
* and the contours.

By default, each contour is represented in a different color.<br>
In this example the data files contains 20 days of observations, this is why the spatial coverage seems very good.

In [ ]:
fig = plt.figure(figsize=(10, 10))
contour2D.add_to_plot(linewidth=2)
data2D.add_to_plot(s=10)
parameters2D.plot_outputgrid()

## Results 

Let's represent the gridded field obtained by the interpolation:

In [ ]:
results2D.add_to_plot?

In [ ]:
fig = plt.figure(figsize=(10, 10))
results2D = pydiva2d.Diva2DResults()
results2D.read_from(DivaFiles.result)
results2D.add_to_plot('analysis', m)
#data2D.add_to_plot(cmap=plt.cm.viridis, s=.2)

The error field is represented using a similar command:

In [ ]:
fig = plt.figure(figsize=(10, 10))
error2D = results2D.add_to_plot('error')
#data2D.add_positions_to_plot(s=2)
plt.colorbar(error2D)
plt.title('Relative error field')

With the select set of parameters, the error is low in the largest part of the basin, the exception is in sub-regions in the vicinity of Greece.

## Finite-element mesh

It is also interesting to display the finite-element mesh that covers our domain of interest.

In [ ]:
mesh2D = pydiva2d.Diva2DMesh()
mesh2D.read_from(DivaFiles.mesh, DivaFiles.meshtopo)

With the `describe` method we obtain details about the mesh:

In [ ]:
mesh2D.describe()

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
mesh2D.add_to_plot(lw=0.1)
plt.title('Finite-element mesh')